In [48]:
import os
import sys
import math
import logging
from pathlib import Path
import json
import glob
import re

import numpy as np
import scipy as sp
import sklearn
import statsmodels.api as sm
from statsmodels.formula.api import ols

%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_context("poster")
sns.set(rc={'figure.figsize': (16, 9.)})
sns.set_style("whitegrid")

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

import sagemaker
import boto3
from IPython.display import HTML
import time
from time import gmtime, strftime
#from misc import get_execution_role, wait_for_s3_object
from sagemaker.pytorch import PyTorch, PyTorchModel
logging.basicConfig(level=logging.INFO, stream=sys.stdout)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
sage_session = sagemaker.session.Session()
s3_bucket = sage_session.default_bucket()  
s3_output_path = 's3://{}/'.format(s3_bucket)
print("S3 bucket path: {}".format(s3_output_path))
aws_region = sage_session.boto_region_name
print("AWS region: {}".format(aws_region))

S3 bucket path: s3://sagemaker-us-west-2-351033349862/
AWS region: us-west-2


**PLEASE** save this file right now using the following naming convention: `NUMBER_FOR_SORTING-YOUR_INITIALS-SHORT_DESCRIPTION`, e.g. `1.0-fw-initial-data-exploration`. Use the number to order the file within the directory according to its usage.

In [50]:
# run in local mode?
local_mode = True

if local_mode:
    instance_type = 'local'
else:
    instance_type = "ml.c5.2xlarge"

In [13]:
try:
    role = sagemaker.get_execution_role()
except:
    role = get_execution_role()

print("Using IAM role arn: {}".format(role))

Using IAM role arn: arn:aws:iam::351033349862:role/service-role/AmazonSageMaker-ExecutionRole-20200227T101932


In [34]:
financial_data_path = Path('../data/preprocessed/stock_contexts_rewards.csv').resolve()
financial_data_path_str = financial_data_path.as_posix()
financial_data_path_str

'/home/ec2-user/SageMaker/space_bandits_deployment/data/preprocessed/stock_contexts_rewards.csv'

In [35]:
financial_data_s3_path = sage_session.upload_data(path=financial_data_path_str, bucket=s3_bucket)

In [52]:
source_dir = Path('../src/space_bandits_deployment').resolve().as_posix()
train_script = Path('../src/space_bandits_deployment/train.py').resolve().as_posix()

In [53]:
hyperparameters = {"action_dim": 8, "context_dim":21}

In [179]:
estimator = PyTorch(entry_point="train.py",source_dir=source_dir,role=role,
                        train_instance_type=instance_type,

                    train_instance_count=1,
                        output_path=s3_output_path,framework_version='1.3.1',hyperparameters=hyperparameters)

In [180]:
estimator.fit(inputs={'train': financial_data_s3_path})

INFO:sagemaker:Creating training-job with name: pytorch-training-2020-03-04-20-04-20-388
INFO:sagemaker.local.image:No AWS credentials found in session but credentials from EC2 Metadata Service are available.
INFO:sagemaker.local.image:docker compose file: 
networks:
  sagemaker-local:
    name: sagemaker-local
services:
  algo-1-42rh5:
    command: train
    environment:
    - AWS_REGION=us-west-2
    - TRAINING_JOB_NAME=pytorch-training-2020-03-04-20-04-20-388
    image: 763104351884.dkr.ecr.us-west-2.amazonaws.com/pytorch-training:1.3.1-cpu-py3
    networks:
      sagemaker-local:
        aliases:
        - algo-1-42rh5
    stdin_open: true
    tty: true
    volumes:
    - /tmp/tmpeewxrvxz/algo-1-42rh5/output/data:/opt/ml/output/data
    - /tmp/tmpeewxrvxz/algo-1-42rh5/output:/opt/ml/output
    - /tmp/tmpeewxrvxz/algo-1-42rh5/input:/opt/ml/input
    - /tmp/tmpeewxrvxz/model:/opt/ml/model
    - /tmp/tmpb7h_3s_h:/opt/ml/input/data/train
version: '2.3'

INFO:sagemaker.local.image:docke

In [160]:
sage_session = sagemaker.local.LocalSession() if local_mode else sage_session


In [181]:
job_name = estimator.latest_training_job.job_name
job_name

'pytorch-training-2020-03-04-20-04-20-388'

In [182]:
if local_mode:
    output_path_prefix = f"{job_name}/output.tar.gz"
    model_path = f"{job_name}/model.tar.gz"
    
else:
    output_path_prefix = f"{job_name}/output/output.tar.gz"
    model_path = f"{job_name}/output/model.tar.gz"
    
#sage_session.download_data(path="../models/output", bucket=s3_bucket, key_prefix=job_name)

In [183]:
model_path = os.path.join(s3_output_path,model_path)
model_path

's3://sagemaker-us-west-2-351033349862/pytorch-training-2020-03-04-20-04-20-388/model.tar.gz'

In [184]:
model=PyTorchModel(model_data=model_path, name="model_v2",
    role=role, framework_version='1.3.1', source_dir=source_dir,entry_point='serve.py')

In [185]:
predictor = model.deploy(initial_instance_count=1, instance_type=instance_type)

INFO:sagemaker:Creating model with name: model_v2
INFO:sagemaker:Creating endpoint with name model_v2
INFO:sagemaker.local.image:serving
INFO:sagemaker.local.image:creating hosting dir in /tmp/tmppslj4lt0
INFO:sagemaker.local.image:No AWS credentials found in session but credentials from EC2 Metadata Service are available.
INFO:sagemaker.local.image:docker compose file: 
networks:
  sagemaker-local:
    name: sagemaker-local
services:
  algo-1-644sp:
    command: serve
    environment:
    - SAGEMAKER_PROGRAM=serve.py
    - SAGEMAKER_SUBMIT_DIRECTORY=s3://sagemaker-us-west-2-351033349862/model_v2/model.tar.gz
    - SAGEMAKER_ENABLE_CLOUDWATCH_METRICS=false
    - SAGEMAKER_CONTAINER_LOG_LEVEL=20
    - SAGEMAKER_REGION=us-west-2
    image: 763104351884.dkr.ecr.us-west-2.amazonaws.com/pytorch-inference:1.3.1-cpu-py3
    networks:
      sagemaker-local:
        aliases:
        - algo-1-644sp
    ports:
    - 8080:8080
    stdin_open: true
    tty: true
    volumes:
    - /tmp/tmpjjwd2m2j:

Exception in thread Thread-13:
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sagemaker/local/image.py", line 614, in run
    _stream_output(self.process)
  File "/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sagemaker/local/image.py", line 673, in _stream_output
    raise RuntimeError("Process exited with code: %s" % exit_code)
RuntimeError: Process exited with code: 1

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sagemaker/local/image.py", line 619, in run
    raise RuntimeError(msg)
RuntimeError: Failed to run: ['docker-compose', '-f', '/tmp/tmppslj4lt0/docker-compose.yaml', 'up', '--build', '--abort-on-container-exit'], Process exited wi

In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type=instance_type)


In [82]:
df = pd.read_csv(financial_data_path)

In [86]:
data = df.to_numpy()

In [88]:
data[0].shape

(29,)

In [91]:
context = data[0,:21]

In [170]:
context

array([-0.424286,  0.880005,  0.5     ,  0.289997,  0.82    ,  0.17    ,
        0.370003,  0.07    ,  0.630001, -0.830001,  0.32    ,  0.119999,
        0.24    ,  0.200001, -0.550003, -0.075   ,  0.059997, -0.130001,
        0.610001,  0.059997,  0.470002])

In [186]:
predictor.predict(context)

algo-1-ufis7_1  | 2020-03-04 20:08:53,863 [INFO ] W-9000-model com.amazonaws.ml.mms.wlm.WorkerThread - Backend response time: 0
algo-1-ufis7_1  | 2020-03-04 20:08:53,864 [INFO ] W-9000-model ACCESS_LOG - /172.18.0.1:45346 "POST /invocations HTTP/1.1" 500 3


OSError: Failed to interpret file <_io.BytesIO object at 0x7fd508f59258> as a pickle

In [177]:
predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: model_v1
INFO:sagemaker:Deleting endpoint with name: model_v1


In [178]:
predictor.delete_model()

INFO:sagemaker:Deleting model with name: model_v1
